### Agentic RAG

In [7]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
temperature = 1.0
max_tokens = 512
model_name = "gpt-3.5-turbo-0125"

In [3]:
llm = ChatOpenAI(
    model=model_name,
    temperature=temperature,
    max_tokens=max_tokens,
    openai_api_key=openai_api_key
)

In [4]:
embeddings = OpenAIEmbeddings(
    openai_api_key = openai_api_key
)
loader = WebBaseLoader("https://docs.smith.langchain.com")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splitted_documents = text_splitter.split_documents(documents)
vectordb = FAISS.from_documents(splitted_documents, embeddings)
retriever = vectordb.as_retriever()

In [5]:
retriever_tool = create_retriever_tool(
    retriever, "langsmith_search",
    "Search for information about Langsmith. For any questions about Langsmith, you must use this tool!")

retriever_tool.name

'langsmith_search'

In [6]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

arxiv.name

'arxiv'

In [8]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

wiki.name

'wikipedia'

In [9]:
from langchain import hub

tools = [wiki, arxiv, retriever_tool]
prompt = hub.pull("hwchase17/openai-functions-agent")

prompt.messages

c:\demonstrations\env\Lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
from langchain.agents import create_openai_tools_agent, AgentExecutor

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True)

In [11]:
question = "Tell me about Langsmith?"

response = agent_executor.invoke({
    "input": question
})

response



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


No good Wikipedia Search Result was found
Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with Lang

{'input': 'Tell me about Langsmith?',
 'output': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed. You can use LangSmith as a standalone platform regardless of how you're creating your LLM applications.\n\nTo get started with LangSmith, you can follow these steps:\n1. Install LangSmith using Python or TypeScript.\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n3. Set up your environment by using the provided commands for Python and TypeScript.\n4. Log your first trace in LangSmith using the LangSmith SDK.\n5. View your trace and explore the results of your evaluation in the LangSmith experiments UI.\n\nFor more information and detailed tutorials, you can visit the LangSmith documentati

In [12]:
question = "what does arxiv paper 1901.00005 talk about?"

response = agent_executor.invoke({
    "input": question
})

response



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1901.00005'}`


Published: 2021-02-23
Title: An Extreme Amplitude, Massive Heartbeat System in the LMC Characterized Using ASAS-SN and TESS
Authors: T. Jayasinghe, K. Z. Stanek, C. S. Kochanek, Todd A. Thompson, B. JThe arXiv paper with the reference number 1901.00005 discusses "An Extreme Amplitude, Massive Heartbeat System in the LMC Characterized Using ASAS-SN and TESS." It is authored by T. Jayasinghe, K. Z. Stanek, C. S. Kochanek, Todd A. Thompson, B. J.

> Finished chain.


{'input': 'what does arxiv paper 1901.00005 talk about?',
 'output': 'The arXiv paper with the reference number 1901.00005 discusses "An Extreme Amplitude, Massive Heartbeat System in the LMC Characterized Using ASAS-SN and TESS." It is authored by T. Jayasinghe, K. Z. Stanek, C. S. Kochanek, Todd A. Thompson, B. J.'}

In [13]:
question = "Machine Learning"

response = agent_executor.invoke({
    "input": question
})

response



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. If you would like to know more specific details or have any other questions related to machine learning, feel free to ask!

> Finished chain.


{'input': 'Machine Learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. If you would like to know more specific details or have any other questions related to machine learning, feel free to ask!'}